<a href="https://colab.research.google.com/github/akash8190/Model-pruning-in-deep-learning-using-keras-and-tensorflow/blob/main/Model_pruning_in_deep_learning_using_Keras_and_Tensorflow_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
 pip install -q tensorflow-model-optimization

     |████████████████████████████████| 174kB 5.9MB/s 


# model prunning using keras and tensorflow using MNIST dataset which is present in keras and see the performance after pruning and before pruning get impactaed or Not

In [4]:
import tempfile
import os

import tensorflow as tf
import numpy as np

from tensorflow import keras

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Train a model for MNIST without pruning


In [16]:
# Load MNIST dataset
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 to 1.
train_images = train_images / 255.0
test_images = test_images / 255.0

# Define the model architecture.
model = keras.Sequential([
  keras.layers.InputLayer(input_shape=(28, 28)),
  keras.layers.Reshape(target_shape=(28, 28, 1)),
  keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation='relu'),
  keras.layers.MaxPooling2D(pool_size=(2, 2)),
  keras.layers.Flatten(),
  keras.layers.Dense(10)
])

# Train the digit classification model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(
  train_images,
  train_labels,
  epochs=4,
  validation_split=0.1,
)

Epoch 1/4
1688/1688 [==============================] - 17s 10ms/step - loss: 0.2994 - accuracy: 0.9149 - val_loss: 0.1143 - val_accuracy: 0.9695
Epoch 2/4
1688/1688 [==============================] - 17s 10ms/step - loss: 0.1151 - accuracy: 0.9666 - val_loss: 0.0767 - val_accuracy: 0.9807
Epoch 3/4
1688/1688 [==============================] - 16s 10ms/step - loss: 0.0819 - accuracy: 0.9760 - val_loss: 0.0665 - val_accuracy: 0.9835
Epoch 4/4
1688/1688 [==============================] - 16s 10ms/step - loss: 0.0671 - accuracy: 0.9801 - val_loss: 0.0613 - val_accuracy: 0.9847


In [18]:
_, baseline_model_accuracy = model.evaluate(
    test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)

_, keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model, keras_file, include_optimizer=False)
print('Saved baseline model to:', keras_file)

Baseline test accuracy: 0.9775999784469604
Saved baseline model to: /tmp/tmp0gu9y51l.h5


# **Fine**-tune pre-trained model with pruning and see the impact on accuracy or **performance**

In [19]:
import tensorflow_model_optimization as tfmot

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Compute end step to finish pruning after 2 epochs.
batch_size = 128
epochs = 2

validation_split = 0.1 # 10% of training set will be used for validation set. 

num_images = train_images.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

# Define model for pruning.
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
}

model_for_pruning = prune_low_magnitude(model, **pruning_params)

# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model_for_pruning.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
prune_low_magnitude_reshape_ (None, 28, 28, 1)         1         
_________________________________________________________________
prune_low_magnitude_conv2d_2 (None, 26, 26, 12)        230       
_________________________________________________________________
prune_low_magnitude_max_pool (None, 13, 13, 12)        1         
_________________________________________________________________
prune_low_magnitude_flatten_ (None, 2028)              1         
_________________________________________________________________
prune_low_magnitude_dense_2  (None, 10)                40572     
Total params: 40,805
Trainable params: 20,410
Non-trainable params: 20,395
_________________________________________________________________


Train and evaluate the model against baseline

In [20]:
logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]

model_for_pruning.fit(train_images, train_labels,
                  batch_size=batch_size, epochs=epochs, validation_split=validation_split,
                  callbacks=callbacks)


Epoch 1/2
422/422 [==============================] - 13s 30ms/step - loss: 0.0915 - accuracy: 0.9749 - val_loss: 0.1164 - val_accuracy: 0.9708
Epoch 2/2
422/422 [==============================] - 12s 29ms/step - loss: 0.1205 - accuracy: 0.9676 - val_loss: 0.0845 - val_accuracy: 0.9797


## For this example, there is minimal loss in test accuracy after pruning, compared to the **baseline**

In [21]:
_, model_for_pruning_accuracy = model_for_pruning.evaluate(
   test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy) 
print('Pruned test accuracy:', model_for_pruning_accuracy)

Baseline test accuracy: 0.9775999784469604
Pruned test accuracy: 0.9690999984741211


# For this example, there is minimal loss in test accuracy after pruning, compared to the **baseline** model

## so we can say that after prunning a little  diffrence in our accuracy a model is light compared to base model so we can say that pruning help to reduce the model complexity and size.